In [97]:
import warnings
warnings.filterwarnings('ignore')
import os
import gc
import sys
import time
import feather
import numpy as np
import pandas as pd
from contextlib import contextmanager
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
import lightgbm as lgbm
import optuna
from sklearn.ensemble import RandomForestRegressor

sys.path.append("/Users/ueda/Desktop/kaggle/mynavi/code/src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage

In [98]:
# ===============
# Constants
# ===============
DATA_DIR = "../input/"
TRAIN_PATH = os.path.join(DATA_DIR, "train.csv")
TEST_PATH = os.path.join(DATA_DIR, "test.csv")

SUB_PATH = os.path.join(DATA_DIR, "sample_submit.csv")
#FOLDS_PATH = "../input/mynavi_Stratifiedfold01.feather"   #289381102       #folds気をつけて
#FOLDS_PATH = "../input/mynavi_Stratifiedfold_access.feather"  #有望

FOLDS_PATH = "../input/mynavi_Stratifiedfold_address.feather" #274592066 最強
#FOLDS_PATH = "../input/mynavi_Stratifiedfold_rent.feather"     #299584731
#FOLDS_PATH = "../input/mynavi_Stratifiedfold_same_build.feather" # 293669458
LOGGER_PATH = "log.txt"
ID_COLUMN = "id"


EXP_ID = "BAYES"

SAVE_PATH = f'../output/{EXP_ID}'
try:
    os.mkdir(SAVE_PATH)
except:
    print("Already directory created")

In [99]:
# ===============
# Settings
# ===============
pd.set_option('display.max_columns', 50)

SEED = 0
N_SPLITS = 5
LOG = False
LGBM_PARAMS = {
    'task': 'train',
    'boosting_type': 'rf',   #rf, gbdt
    'objective': 'poisson',    #regression, gamma, poisson gamma早すぎ
    'metric': "rmse",
    "bagging_freq": 1,
    'learning_rate': 0.05,
    'max_depth': 8,            #7,9 < 8 240690458
    'num_leaves': int(.7 * 8 ** 2),        #15 < int()
    'subsample': 0.9,
    'subsample_freq': 1,
    'colsample_bytree': 0.4, # 0.4, 0.7, 0.8で271219355 0.8良さげ
    'max_bin': 255,
    'reg_alpha': 0.1,        # 0.1
    'reg_lambda': 0.1,
    'min_child_samples': 10,   #20 vs 10
    'min_gain_to_split': 0.02,
    'min_data_in_bin': 3,
    'bin_construct_sample_cnt': 5000,
    'cat_l2': 5,    #5でも10でも
    'verbose': -1,
    'nthread': -1,
    'seed': SEED,
}
LGBM_FIT_PARAMS = {
    'num_boost_round': 50000,
    'early_stopping_rounds': 100,
    'verbose_eval': 5000,
}

use_cols = []

drop_columns = ['fold_id', 'id', 'キッチン', 'バス・トイレ', '周辺環境',
                '室内設備', '所在階', '放送・通信', '間取り',
                '築年数', '賃料', '面積', '駐車場', "路線_0", "駅_0", "方角", #"所在地", #"same_build",
                 "路線_1", "駅_1", "路線_2", "駅_2", "契約期間", '建物構造', 'アクセス', #"地域_n丁目", "地域名",# "catuse_駅_0", "catuse_路線_0"
]

#ategorical_features = ["catuse_駅_0", "catuse_路線_0", #"地域名",
#                        "cat_int間取り", "cat_building_height",'所在_区'
#]

categorical_features = ["catuse_駅_0", "catuse_路線_0", "所在地", "地域名", "地域_n丁目",
                        "cat_int間取り", "cat_building_height",'所在_区',
] #258442952

#TE_columns = ["所在_区", "間取りtype", 'catuse_路線_0', 'catuse_駅_0',
#           "cat_int間取り", "cat_building_height", "地域名", '地域_n丁目', "same_build"
#]
#drop_columns = drop_columns + [col for col in categorical_features if col in TE_columns]
#categorical_features = [cat_col for cat_col in categorical_features if cat_col not in TE_columns]

FEATURES = [
    "../code/feature_csv/base_feature1.feather",
    "../code/feature_csv/nonleak_group_feature2.feather",
    #'../code/feature_csv/nonleak_nearStation_group_feature.feather',
    #'../code/feature_csv/near_access_feature.feather',
    "../code/feature_csv/word_contain_sparse_feature.feather",       #ないほうがいいけど悪くない
    
    "../code/feature_csv/access_feature.feather",
    #"../code/feature_csv/access_feature+GMM.feather",
    
    "../code/feature_csv/address_latlng.feather",
    #"../code/feature_csv/meta_features.feather",   #有効
    #"../code/feature_csv/out.feather",
    #"../code/feature_csv/tika_feature.feather",     #使う前195
    "../code/feature_csv/out_2.feather",           #198
    "../code/feature_csv/eki_people.feather",       #有効
    "../code/feature_csv/address_from_center.feather" #181
]
setup_logger(out_file=LOGGER_PATH)

2019-11-07 23:15:09,640 - INFO - logger set up


<RootLogger root (DEBUG)>

In [100]:
# ===============
# Functions
# ===============
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')
    
def load_feature_feather(df, filename):
    feature = feather.read_dataframe(filename)
    df = pd.concat((df, feature), axis=1)
    del feature
    gc.collect()
    return df

def load_feature_csv(df, filename):
    feature = pd.csv(filename)
    df = pd.concat((df, feature), axis=1)
    del feature
    gc.collect()
    return df

def modify_rent(target, df_input):
    """ id == 5776 で賃料の0が一つ多い"""
    target = np.where(df_input["id"]==5776, 120350, target)
    
    return target


"""def TE_feature(df_main, df_add, TE_columns = TE_columns):
    df_out = pd.DataFrame()
    add_TE_cols = [f"TE_{col}_mul_面積" for col in TE_columns]
    tmp_cols = [f"TE_{col}" for col in TE_columns]
    TE_df = df_add[TE_columns]
    TE_df.columns = tmp_cols
    
    TE_mul_df = pd.DataFrame(TE_df[tmp_cols].values * df_main["float面積"].values.reshape(-1,1), columns=add_TE_cols)
    TE_mul_df.index = df_main.index
    TE_df.index = df_main.index
    
    TE_mul_df["std_TE賃料"] = TE_mul_df.std(axis=1)
    TE_mul_df["min_TE賃料"] = TE_mul_df.min(axis=1)
    TE_mul_df["max_TE賃料"] = TE_mul_df.max(axis=1)
    TE_mul_df["mean_TE賃料"] = TE_mul_df.mean(axis=1)
    
    df_out = pd.concat([TE_df, TE_mul_df], axis=1)
    
    return df_out"""

def calc_loss(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

## main

In [101]:
with timer("load data"):
    train = feather.read_dataframe(FOLDS_PATH)
    test = pd.read_csv(TEST_PATH)
    if LOG:
        y = np.log1p(train['賃料'].copy())
    else:
        y = train['賃料'].copy()
    folds = train[["fold_id"]].copy()
    n_train = len(train)
    train = train.append(test).reset_index(drop=True)
    del test
    gc.collect()

2019-11-07 23:15:11,304 - INFO - [load data] done in 1 s


In [102]:
with timer("load featuers"):
    for f in FEATURES:
        train = load_feature_feather(train, f)
        
    #train["same_build"] = train["building_floor"].astype(str) + train["地域_n丁目"] + train["int築年"].astype(str)

2019-11-07 23:15:20,803 - INFO - [load featuers] done in 9 s


In [103]:
with timer("preprocessing"):
    for c in categorical_features:
        lbl = LabelEncoder()
        lbl.fit(list(train[c].astype("str").values))
        train[c] = lbl.transform(list(train[c].astype("str").values))

    
    """記入ミスの修正"""
    if "id" in train.columns:
        y = modify_rent(y, train[:n_train])
        print("Modifify Done")
        print("****************")
        
    
    
    try:
        train.drop(drop_columns, axis=1, inplace=True)
        print("DROP DONE!")
        print("****************")
    except:
        pass
    
    
    """importtanceで区切るなら"""
    if len(use_cols)==0:
        test = train[n_train:]
        train = train[:n_train]
    
    elif len(use_cols)>0:
        test = test[use_cols]
        train = train[use_cols]
        
    try:
        train.drop(drop_columns, axis=1, inplace=True)
    except:
        pass
    
    
    """外れ値分類の特量追加"""
    """out = np.percentile(y, [85])
    high_out = np.percentile(y, [99.8])
    outlin = np.where(y > out,1,0)
    outlin = np.where(y > high_out, 2, outlin)
    model = lgbm.LGBMClassifier().fit(train, outlin)
    train["out"] = outlin
    test["out"] = model.predict(test)"""
    
    
    train = reduce_mem_usage(train)
    train.to_feather(f"../features/{EXP_ID}_df.feather")
        
    #gc.collect()
    #target_test = pd.read_csv(f'../code/feature_csv/target_encoding_test.csv')
    #test = pd.concat([test, TE_feature(test, target_test)], axis=1)
    
    features = list(test.columns.values)

Modifify Done
****************
DROP DONE!
****************
Memory usage of dataframe is 237.70 MB
column =  990
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
Memory usage after optimization is: 99.64 MB
Decreased by 58.1%


2019-11-07 23:16:02,130 - INFO - [preprocessing] done in 41 s


In [104]:
tr = train.dropna(how='any', axis=1).columns
ts = test.dropna(how='any', axis=1).columns
[c for c in tr if c not in ts]

['float面積std_of_所在_区',
 '築月数std_of_所在_区',
 'contract_periodstd_of_所在_区',
 'building_floorstd_of_所在_区',
 'living_floorstd_of_所在_区',
 '設備和std_of_所在_区',
 '間取りtypestd_of_所在_区',
 'int間取りstd_of_所在_区',
 '駅からの距離_0std_of_所在_区',
 '駅からの距離_1std_of_所在_区',
 'int建物構造std_of_所在_区',
 '駅_0からの距離_std']

In [105]:
ts_drop = ['駅からの距離_2mean_of_int築年', '駅からの距離_2min_of_int築年', '駅からの距離_2max_of_int築年']
tr_drop = ['float面積std_of_所在_区',
 '築月数std_of_所在_区',
 'contract_periodstd_of_所在_区',
 'building_floorstd_of_所在_区',
 'living_floorstd_of_所在_区',
 '設備和std_of_所在_区',
 '間取りtypestd_of_所在_区',
 'int間取りstd_of_所在_区',
 '駅からの距離_0std_of_所在_区',
 '駅からの距離_1std_of_所在_区',
 'int建物構造std_of_所在_区',
 '駅_0からの距離_std']

In [106]:
"""tr_drop_col = ['float面積std_of_所在_区',
            '築月数std_of_所在_区','contract_periodstd_of_所在_区','building_floorstd_of_所在_区','living_floorstd_of_所在_区',
            '設備和std_of_所在_区','間取りtypestd_of_所在_区','int間取りstd_of_所在_区','駅からの距離_0std_of_所在_区',
            '駅からの距離_1std_of_所在_区','int建物構造std_of_所在_区','駅_0からの距離_std', '駅からの距離_2mean_of_int築年', '駅からの距離_2min_of_int築年', '駅からの距離_2max_of_int築年'
]

ts_drop_col = ['駅からの距離_2mean_of_int築年', '駅からの距離_2min_of_int築年', '駅からの距離_2max_of_int築年']

tsss_col = ['float面積std_of_所在_区','築月数std_of_所在_区','contract_periodstd_of_所在_区','building_floorstd_of_所在_区',
            'living_floorstd_of_所在_区','設備和std_of_所在_区','間取りtypestd_of_所在_区','int間取りstd_of_所在_区',
            '駅からの距離_0std_of_所在_区','駅からの距離_1std_of_所在_区','int建物構造std_of_所在_区','駅_0からの距離_std'
           ]"""

"tr_drop_col = ['float面積std_of_所在_区',\n            '築月数std_of_所在_区','contract_periodstd_of_所在_区','building_floorstd_of_所在_区','living_floorstd_of_所在_区',\n            '設備和std_of_所在_区','間取りtypestd_of_所在_区','int間取りstd_of_所在_区','駅からの距離_0std_of_所在_区',\n            '駅からの距離_1std_of_所在_区','int建物構造std_of_所在_区','駅_0からの距離_std', '駅からの距離_2mean_of_int築年', '駅からの距離_2min_of_int築年', '駅からの距離_2max_of_int築年'\n]\n\nts_drop_col = ['駅からの距離_2mean_of_int築年', '駅からの距離_2min_of_int築年', '駅からの距離_2max_of_int築年']\n\ntsss_col = ['float面積std_of_所在_区','築月数std_of_所在_区','contract_periodstd_of_所在_区','building_floorstd_of_所在_区',\n            'living_floorstd_of_所在_区','設備和std_of_所在_区','間取りtypestd_of_所在_区','int間取りstd_of_所在_区',\n            '駅からの距離_0std_of_所在_区','駅からの距離_1std_of_所在_区','int建物構造std_of_所在_区','駅_0からの距離_std'\n           ]"

In [107]:
train = train.replace([np.inf, -np.inf], np.nan)
test = test.replace([np.inf, -np.inf], np.nan)

train_ = train.dropna(how='any', axis=1).drop(tr_drop, axis=1)
test_ = test.dropna(how='any', axis=1).drop(ts_drop, axis=1)

In [108]:
from sklearn.linear_model import BayesianRidge, LinearRegression, Ridge

forest = BayesianRidge()
forest.fit(train_, y)

# 評価
#print('Train score: {}'.format(forest.score(train_, y)))
#print('Test score: {}'.format(forest.score(test_, y_test)))
#print('f1 score: {:.3f}'.format(f1_score(y_test, forest.predict(test))))

BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
              fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
              normalize=False, tol=0.001, verbose=False)

In [109]:
y_test = forest.predict(test_)
y_train = forest.predict(train_)

In [110]:
#yof = pd.read_csv(os.path.join(SAVE_PATH,f"yoof_and_target.csv"))
yof[EXP_ID] = y_train

yof.to_csv(f"../output/vs_trains/{EXP_ID}.csv", index=False)

FileNotFoundError: [Errno 2] File b'../output/BAYES/yoof_and_target.csv' does not exist: b'../output/BAYES/yoof_and_target.csv'

In [111]:
with timer("sub"):
    sub = pd.read_csv(SUB_PATH, header=None)
    sub[1] = y_test
    LOGGER.info(f'len sub={len(sub)}')
    sub.to_csv(os.path.join(SAVE_PATH,f'{EXP_ID}.csv'), index=False, header=None)

2019-11-07 23:16:45,518 - INFO - len sub=31262
2019-11-07 23:16:45,626 - INFO - [sub] done in 0 s


In [112]:
y_train

array([ 81408.04811752,  26447.552148  , 113151.75797998, ...,
       127539.11381202,  92780.591588  ,  76721.58074364])

In [88]:
"""importtances visualization"""
cols = importances[["feature", "gain"]].groupby("feature").mean().sort_values(
    by="gain", ascending=False)[:1000].index

best_features = importances.loc[importances.feature.isin(cols)]
plt.figure(figsize=(20,20))
sns.barplot(x="gain", y="feature", data=best_features.sort_values(by="gain", ascending=False))
plt.title(f'LightGBM Features (avg over folds)_{EXP_ID}')
plt.tight_layout()
plt.savefig(os.path.join(SAVE_PATH,(f'lgbm_importances_{EXP_ID}.png')))

NameError: name 'importances' is not defined

In [34]:
categorical_features
#TE_feature[TE_columns]

['catuse_駅_0',
 'catuse_路線_0',
 '所在地',
 '地域名',
 '地域_n丁目',
 'cat_int間取り',
 'cat_building_height',
 '所在_区']

In [35]:
pd.set_option('display.max_rows', 500)
importances.sort_values("gain", ascending=False)

,feature,gain,split,fold
885,out,6.168878e+09,34,4
45,間取りtype,2.042667e+09,16,4
167,平均float面積_diff_一戸建て,1.469182e+09,81,4
24,所在_区,1.194393e+09,119,4
127,float面積mean_of_間取りtype,1.032701e+09,10,4
130,float面積max_of_間取りtype,8.410872e+08,6,4
168,平均float面積_percent_一戸建て,7.448346e+08,33,4
132,平均float面積_percent_間取りtype,6.835783e+08,52,4
16,float面積,6.204885e+08,77,4
128,float面積std_of_間取りtype,5.028850e+08,3,4


In [36]:
np.percentile(y, [99.8])
#(y>805900).sum()

array([650000.])

In [21]:
use_cols = list(importances[importances["gain"]>30000]["feature"].values)
print(len(use_cols), len(train.columns))
use_cols = [col for col in use_cols if col in train.columns]
use_cols = use_cols + [col for col in categorical_features if col not in use_cols]

291 886


In [22]:
yoof_and_target[yoof_and_target["賃料"] == yoof_and_target["賃料"].max()]

,賃料,exp0_+address_poisson_outlin_3class_RF
5093,2500000,9.009640e+06


In [154]:
out = np.percentile(y, [85])
high_out = np.percentile(y, [99.8])

test_out = pd.DataFrame(y_test)
test_out["out"] = np.where(test_out[0] > int(out), 1, 0)
test_out["out"] = np.where(test_out[0] > int(high_out), 2, test_out["out"])

In [158]:
#output_dir = '../code/feature_csv/'
#pd.concat([train[["out"]], test_out[["out"]]], axis=0).reset_index(drop=True).to_feather(os.path.join(output_dir, 'out.feather'))


In [130]:
out = np.percentile(y, [85])
high_out = np.percentile(y, [99.8])
outlin = np.where(y > out,1,0)
outlin = np.where(y > high_out, 2, outlin)
model = lgbm.LGBMClassifier().fit(train, outlin)
train["out"] = outlin
test["out"] = model.predict(test)

In [121]:
train["out"].value_counts()

array([650000.])

In [107]:
test["out"].value_counts()

0    26496
1     4604
2      162
Name: out, dtype: int64

# ENSEMBLE

In [ ]:
x_train[x_train["max_TE賃料"]==x_train["max_TE賃料"].min()]

In [ ]:
test2 = test.reset_index()
[test2["out"]==1]

In [ ]:
sub

In [ ]:
up_model = pd.read_csv("../output/exp1_poisson_upper100000/exp1_poisson_upper100000_100000.csv", header=None)

In [ ]:
ens = sub[[0]]
ens["1"] = (np.where([test2["out"]==1], up_model[1], sub[1])).reshape(-1,1)

In [ ]:
ens.to_csv('ens.csv', index=False, header=None)